In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim

from utils import seed

seed.seed_everything()

In [2]:
# (already downloaded) dataset path 
path = './data/data'

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [3]:
from model.backbone import VGG16, ResNet34
from model.simclr import basic_simclr

backbone1 = ResNet34.ResNet34_Backbone(pretrain=False).to(device=device)
model1 = basic_simclr.BasicSimCLR(backbone1).to(device=device)

backbone2 = VGG16.VGG16_Backbone(pretrain=False).to(device=device)
model2 = basic_simclr.BasicSimCLR(backbone2).to(device=device)

# print(model)

c:\anaconda\envs\wsl\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\anaconda\envs\wsl\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [4]:
# default hyperparameters
batch_size = 16
lr = 3e-4
weight_decay = 1e-3

In [5]:
from model.simclr import data_aug

dataset = data_aug.ContrastiveLearningDataset(path=path, input_shape=96)

train_set = dataset.get_dataset()
train_loader = torch.utils.data.DataLoader(train_set, 
                                           batch_size=batch_size, 
                                           shuffle=True, 
                                           num_workers=0,
                                           pin_memory=True, 
                                           drop_last=True)

In [9]:
optimizer = torch.optim.Adam(model1.parameters(), lr, weight_decay=weight_decay)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 
                                                       T_max=len(train_loader), 
                                                       eta_min=0,
                                                       last_epoch=-1)

In [10]:
from model.simclr import simclr_train

simclr = simclr_train.SimCLR(model=model1,
                             model_name='VGG16',
                             scheduler=scheduler,
                             optimizer=optimizer,
                             device=device,
                             epochs=1)

In [ ]:
simclr.train(train_loader=train_loader)